In [1]:
from bs4 import BeautifulSoup as bs
import urllib.request as r
import pandas as pd
import sys
import time


In [2]:
# This function will swap the rows and columns of a 2x2 list
# It uses python's inbuilt zip() function to handle transposition
# However zip() returns a list of tuples instead, so this function will convert that back into a list of lists
def transpose(lol):
    transposedToLot = zip(*lol) #the * asterisk is very important! It prevents zip() from creating a list with a single tuple of tuples
    convertedToLol = [list(x) for x in transposedToLot]
    return list(convertedToLol)


In [3]:
a = "https://www.wunderground.com/history/airport/RCSS/2017/"
b = "/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968"

urls = [] #contains all the urls that we will be accessing

# this loop will populate the url list with the relevant urls
for month in range(5,12):
    if (month==6 or month==9 or month==11):
        for day in range(1,31):
            urls.append(a + str(month) + "/" + str(day) + b)
    else:
        for day in range(1,32):
            urls.append(a + str(month) + "/" + str(day) + b)
print("URLs generated. There %s %d url%s" % ("is" if len(urls)==1 else "are", len(urls), "" if len(urls)==1 else "s."))
urls

URLs generated. There are 214 urls.


['https://www.wunderground.com/history/airport/RCSS/2017/5/1/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968',
 'https://www.wunderground.com/history/airport/RCSS/2017/5/2/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968',
 'https://www.wunderground.com/history/airport/RCSS/2017/5/3/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968',
 'https://www.wunderground.com/history/airport/RCSS/2017/5/4/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968',
 'https://www.wunderground.com/history/airport/RCSS/2017/5/5/DailyHistory.html?req_city=New+Taipei+City&req_state=TPQ&req_statename=Taiwan&reqdb.zip=00000&reqdb.magic=5&reqdb.wmo=58968',
 'https://www.wunderground.com/history/airport/RCSS/2017/5/6/Dail

In [4]:
#beautifulsoup boilerplate code
opener = r.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]


In [5]:
df = pd.DataFrame() #Initialize the dataframe
j = 1
mth = 5
dy = 1
for item in urls:
    html = opener.open(item)
    bsObj = bs(html.read(), "html5lib") #bsObj is a BeautifulSoup Object that contains the entire html file data
    bs_table = bsObj.find("table", {"id":"obsTable"}) #bs_table is a table with the id 'obsTable', extracted from bsObj
    tbl_list = []

    #Go through each bs_table cell with the tag <tr> and copy it into a 2x2 table 'tbl_list'
    for row in bs_table.findAll("tr"):
        tmp_list = [] #tmp_list represents the current row we are building
        for item in row:
            try:
                string = item.get_text().strip()
                s=""
                for c in string:
                    if c.isdigit() or c.isalpha() or c==r"." or c==r" " or c=="(" or c==")" or c==":": #only these characters are allowed
                        s += c
                    else:
                        break #the moment you encounter an invalid character, stop reading and move on to the next item in the row
                tmp_list.append(s)
            except:
                pass
        tbl_list.append(tmp_list)

    # temporarily transpose the columns to become rows so that I can remove the irrelevant columns
    tbl_list = transpose(tbl_list)

    newlist = [] #Because python is acting fucked up, instead of removing all columns that are not "Time", "Temp" or "Humidity" I have to create a new list and append the "Time", "Temp" & "Humidity" columns to it
    #while tbl_list contains every column, newlist only contains the "Time (CST)", "Temp." and "Humidity" columns
    for i in tbl_list:
        if (i[0] == r"Time (CST)" or i[0] == r"Temp." or i[0] == r"Humidity"):
            newlist.append(i)

    #add the corresponding date to every time entry
    newlist[0] = ["2017/" + str(mth) + "/" + str(dy) + " " + time if time != "Time (CST)" else time for time in newlist[0]]
    #and then increment the date counter
    if (mth==6 or mth==9 or mth==11):
        if (dy>=30):
            mth += 1
            dy = 1
        else:
            dy += 1
    else:
        if (dy>=31):
            mth += 1
            dy = 1
        else:
            dy += 1

    newlist = transpose(newlist) #transpose it back
    df=df.append(pd.DataFrame(newlist)) #append the table for this url page to the overall dataframe
    print(str(j) + "/" + str(len(urls))) #progress counter
    j += 1
#for loop ends here


1/214
2/214
3/214
4/214
5/214
6/214
7/214
8/214
9/214
10/214
11/214
12/214
13/214
14/214
15/214
16/214
17/214
18/214
19/214
20/214
21/214
22/214
23/214
24/214
25/214
26/214
27/214
28/214
29/214
30/214
31/214
32/214
33/214
34/214
35/214
36/214
37/214
38/214
39/214
40/214
41/214
42/214
43/214
44/214
45/214
46/214
47/214
48/214
49/214
50/214
51/214
52/214
53/214
54/214
55/214
56/214
57/214
58/214
59/214
60/214
61/214
62/214
63/214
64/214
65/214
66/214
67/214
68/214
69/214
70/214
71/214
72/214
73/214
74/214
75/214
76/214
77/214
78/214
79/214
80/214
81/214
82/214
83/214
84/214
85/214
86/214
87/214
88/214
89/214
90/214
91/214
92/214
93/214
94/214
95/214
96/214
97/214
98/214
99/214
100/214
101/214
102/214
103/214
104/214
105/214
106/214
107/214
108/214
109/214
110/214
111/214
112/214
113/214
114/214
115/214
116/214
117/214
118/214
119/214
120/214
121/214
122/214
123/214
124/214
125/214
126/214
127/214
128/214
129/214
130/214
131/214
132/214
133/214
134/214
135/214
136/214
137/214
138/214
139/

In [6]:
#setting the header and column index
df.columns = df.iloc[0] #set the first row as the header row (which determines the column names)
df = df.drop_duplicates(subset="Time (CST)", keep=False) #delete all the useless duplicate header rows ('Time', 'Temp' & 'Humidity')in the table data
df = df.set_index("Time (CST)") #set the first column as index
df

,Temp.,Humidity
Time (CST),,
2017/5/1 12:00 AM,22.0,73
2017/5/1 1:00 AM,21.0,78
2017/5/1 2:00 AM,21.0,78
2017/5/1 3:00 AM,21.0,78
2017/5/1 4:00 AM,20.0,78
2017/5/1 5:00 AM,20.0,78
2017/5/1 6:00 AM,21.0,73
2017/5/1 6:30 AM,22.0,73
2017/5/1 7:00 AM,24.0,69


In [7]:
#getting the hourly average
df.index=pd.DatetimeIndex(df.index) #convert date index datatype to DateTime
df.index=df.index.map(lambda x:x.replace(minute=0)) #normalize all time to hours, e.g 6.30PM becomes 6.00PM
                                                    #this will introduce duplicates datetime entries
df['Temp.'] = df['Temp.'].apply(pd.to_numeric, errors='ignore') #I can use to_numeric to convert a column of strings to numbers,
df['Humidity'] = df['Humidity'].apply(lambda x: int(x)) #Or I can use a simple lamba function to do the same
df = df.groupby('Time (CST)').mean() #aggregate the duplicate entries by taking their mean
df['Humidity'] = df['Humidity'].apply(lambda x: int(x)) #mean() converts humidity column data to floats, so convert them back to int

all_days = pd.date_range(df.index.min(), df.index.max(), freq='H')
df = df.reindex(all_days).interpolate()


#df.columns = ["Temp/°C", "Humidity/%"] #Add units to column headers
df.columns = ["temp", "%humidity"]
df.index.name = 'ts'
df.head

<bound method NDFrame.head of 0                        Temp.  Humidity
Time (CST)                              
2017-05-01 00:00:00  22.000000        73
2017-05-01 01:00:00  21.000000        78
2017-05-01 02:00:00  21.000000        78
2017-05-01 03:00:00  21.000000        78
2017-05-01 04:00:00  20.000000        78
2017-05-01 05:00:00  20.000000        78
2017-05-01 06:00:00  21.500000        73
2017-05-01 07:00:00  24.500000        67
2017-05-01 08:00:00  26.500000        63
2017-05-01 09:00:00  28.500000        51
2017-05-01 10:00:00  30.000000        42
2017-05-01 11:00:00  31.000000        46
2017-05-01 12:00:00  31.000000        47
2017-05-01 13:00:00  30.000000        53
2017-05-01 14:00:00  30.500000        51
2017-05-01 15:00:00  30.000000        46
2017-05-01 16:00:00  29.000000        51
2017-05-01 17:00:00  28.000000        58
2017-05-01 18:00:00  27.000000        61
2017-05-01 19:00:00  25.500000        67
2017-05-01 20:00:00  25.000000        69
2017-05-01 21:00:00  24.500

In [8]:
df.info()
#May-Nov has (31 + 30 + 31 + 31 + 30 + 31 + 30)(24) = 5136 hours in total. There should be 5136 entries

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5112 entries, 2017-05-01 00:00:00 to 2017-11-30 23:00:00
Data columns (total 2 columns):
Temp.       5112 non-null float64
Humidity    5112 non-null int64
dtypes: float64(1), int64(1)
memory usage: 119.8 KB
